In [1]:
import pandas as pd

In [5]:
res_df1 = pd.read_csv("synth_results_many.csv")

In [6]:
res_df = pd.read_csv("synth_results_w_recon.csv")

In [7]:
res_df2 = pd.read_csv("synth_results_w_recon_normalize.csv")

In [4]:
# res_df.sort_values(['seedmethod', 'bcv', 'dropout']).to_csv('synth_results_w_recon_sorted.csv')

In [8]:
res_df

,seedmethod,bcv,dropout,demap,accuracy,recon score
0,"groups,46",0.2,0.7,0.746910,0.750408,0.624526
1,"groups,46",0.2,0.7,0.718545,0.748234,-0.009220
2,"paths,44",0.4,0.5,0.588830,0.724110,-0.029067
3,"paths,44",0.4,0.5,0.579844,0.730167,0.402261
4,"groups,43",0.6,0.2,0.583970,0.715159,-0.013807
...,...,...,...,...,...,...
175,"groups,42",0.2,0.5,0.777820,0.747658,0.678843
176,"paths,44",0.4,0.7,0.569473,0.727922,0.346185
177,"groups,43",0.6,0.7,0.545900,0.701400,-0.013308
178,"groups,43",0.6,0.7,0.468422,0.692136,0.270351


In [9]:
res_df2

,seedmethod,bcv,dropout,demap,accuracy,recon score,reconstr_weight
0,"groups,46",0.2,0.7,0.744974,0.751601,0.036525,0.0
1,"groups,46",0.2,0.7,0.726728,0.755885,0.650285,0.1
2,"paths,44",0.4,0.5,0.586015,0.716610,-0.021480,0.0
3,"paths,44",0.4,0.5,0.591137,0.726693,0.414529,0.1
4,"groups,43",0.6,0.2,0.558332,0.707529,-0.001090,0.0
...,...,...,...,...,...,...,...
175,"paths,44",0.4,0.7,0.557752,0.715845,0.023779,0.0
176,"paths,44",0.4,0.7,0.562843,0.711590,0.369864,0.1
177,"groups,42",0.2,0.2,0.773730,0.766645,-0.027547,0.0
178,"groups,43",0.6,0.5,0.597661,0.699193,0.346553,0.1


In [5]:
# res_df1.sort_values(['dataset', 'bcv', 'dropout']).to_csv('syth_results_many_sorted.csv')

In [10]:
res_df_merge1 = pd.concat([res_df, res_df1['recon_weights']], axis=1)

In [11]:
res_df_merge2 = pd.concat([res_df2, res_df1['recon_weights']], axis=1)

In [12]:
res_df_merge1['recon_weights'] = res_df_merge1['recon_weights'].apply(lambda x: (x == '[0.9, 0.1, 0]'))
res_df_merge1.rename(columns={'recon_weights': 'Reconstruction Loss'}, inplace=True)

In [13]:
res_df_merge2['recon_weights'] = res_df_merge2['recon_weights'].apply(lambda x: (x == '[0.9, 0.1, 0]'))
res_df_merge2.rename(columns={'recon_weights': 'Reconstruction Loss'}, inplace=True)

In [14]:
res_df_merge1.sort_values(['seedmethod', 'bcv', 'Reconstruction Loss'], inplace=True)

In [15]:
res_df_merge2.sort_values(['seedmethod', 'bcv', 'Reconstruction Loss'], inplace=True)

In [16]:
cols = ['seedmethod', 'bcv', 'dropout', 'Reconstruction Loss', 'demap', 'accuracy', 'recon score']
res_df_merge1 = res_df_merge1[cols]
res_df_merge2 = res_df_merge2[cols]

In [18]:
import pandas as pd

mean_df = res_df_merge1.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).mean().round(3)
std_df = res_df_merge1.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).std().round(3)

# Merge the mean and std dataframes
combined_df = mean_df.merge(std_df, left_index=True, right_index=True, suffixes=('_mean', '_std'))

# Format the results into a single column for each feature
for col in mean_df.columns:  # Iterate directly through the columns
    combined_df[col] = combined_df[col+'_mean'].astype(str) + "(±" + combined_df[col+'_std'].astype(str) + ")"
    # Since we're directly modifying the combined_df, no need to drop the columns in this loop

# Drop the original mean and std columns after processing
combined_df1 = combined_df.loc[:, ~combined_df.columns.str.contains('_mean|_std')]

# combined_df will now have the desired format
print(combined_df1)

                                         demap       accuracy     recon score
bcv dropout Reconstruction Loss                                              
0.2 0.2     False                 0.706(±0.09)   0.772(±0.03)  -0.006(±0.029)
            True                 0.705(±0.087)  0.775(±0.028)   0.672(±0.066)
    0.5     False                0.688(±0.095)   0.774(±0.03)    0.006(±0.03)
            True                 0.706(±0.089)  0.776(±0.023)   0.645(±0.066)
    0.7     False                0.675(±0.094)  0.766(±0.031)   0.002(±0.022)
            True                 0.678(±0.096)  0.768(±0.029)   0.594(±0.078)
0.4 0.2     False                0.622(±0.125)  0.748(±0.033)    -0.0(±0.022)
            True                 0.626(±0.128)   0.753(±0.03)   0.469(±0.093)
    0.5     False                0.613(±0.128)  0.752(±0.036)   -0.003(±0.02)
            True                 0.608(±0.126)   0.75(±0.033)    0.447(±0.09)
    0.7     False                 0.59(±0.138)  0.738(±0.026)  -

In [19]:
import pandas as pd

mean_df = res_df_merge2.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).mean().round(3)
std_df = res_df_merge2.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).std().round(3)

# Merge the mean and std dataframes
combined_df = mean_df.merge(std_df, left_index=True, right_index=True, suffixes=('_mean', '_std'))

# Format the results into a single column for each feature
for col in mean_df.columns:  # Iterate directly through the columns
    combined_df[col] = combined_df[col+'_mean'].astype(str) + "(±" + combined_df[col+'_std'].astype(str) + ")"
    # Since we're directly modifying the combined_df, no need to drop the columns in this loop

# Drop the original mean and std columns after processing
combined_df2 = combined_df.loc[:, ~combined_df.columns.str.contains('_mean|_std')]

# combined_df will now have the desired format
print(combined_df2)

                                         demap       accuracy    recon score
bcv dropout Reconstruction Loss                                             
0.2 0.2     False                 0.67(±0.098)   0.76(±0.028)  0.191(±0.328)
            True                 0.714(±0.073)  0.778(±0.033)  0.403(±0.364)
    0.5     False                0.682(±0.086)  0.767(±0.024)  0.212(±0.325)
            True                 0.729(±0.075)  0.781(±0.023)  0.512(±0.279)
    0.7     False                0.661(±0.076)   0.761(±0.02)     0.4(±0.32)
            True                 0.719(±0.093)   0.77(±0.029)  0.208(±0.319)
0.4 0.2     False                 0.65(±0.116)   0.748(±0.02)  0.285(±0.259)
            True                 0.598(±0.126)  0.737(±0.033)  0.201(±0.245)
    0.5     False                0.615(±0.131)  0.731(±0.039)   0.13(±0.206)
            True                 0.593(±0.108)  0.747(±0.023)  0.356(±0.214)
    0.7     False                0.585(±0.149)   0.73(±0.025)  0.055(±0.133)

In [20]:
combined_df.to_csv('synth_res_final.csv', index=False)